# Inputting game stats
-note the data is 2016

In [9]:
import numpy as np

class GameStats(object):          
    
    def __init__(self, homeTeamNameIndex, homeTeamScoreIndex, homeTeamStatsIndex, visitorTeamNameIndex, visitorTeamScoreIndex, visitorTeamStatsIndex):
        #parse the text file
        self.statsFile = open("baseball2016.txt", "r")
        self.topArray = []
        self.sideArray = []  
        self.sc = np.zeros((30,30,30), np.int32) 
        self.sc[:,:,:] = -1  
        self.am = np.zeros((30,30), np.float32)
        self.gameList = []
        
        seenTeams = []
        for line in self.statsFile:
            token = line.split(',')  #tokenize the string
            tokenIndex = [homeTeamNameIndex, homeTeamScoreIndex, visitorTeamNameIndex, visitorTeamScoreIndex] + [i for i in homeTeamStatsIndex] + [i for i in visitorTeamStatsIndex]
            attributes = dict()
            
            for i in range(len(token)):
                if(i in tokenIndex):
                    attributes[i] = self.removeQuotes(token[i])
                        
            self.addScore(attributes[homeTeamNameIndex], attributes[visitorTeamNameIndex], attributes[homeTeamScoreIndex], attributes[visitorTeamScoreIndex])                
            self.addGame(attributes[homeTeamNameIndex], attributes[homeTeamScoreIndex], [attributes[i] for i in homeTeamStatsIndex], attributes[visitorTeamNameIndex], attributes[visitorTeamScoreIndex], [attributes[i] for i in visitorTeamStatsIndex])
            
            if(attributes[homeTeamNameIndex] not in seenTeams):
                seenTeams.append(attributes[homeTeamNameIndex])
            
        self.buildAvgMatrix()
        self.statsFile.close()
        #self.gameList = [bin(2**i)[2:].zfill(len(seenTeams)) if x == seenTeams[i] else x for i in range(len(seenTeams)) for x in self.gameList]
        # take the teams and convert the indexes of the order that they appeared in into the hot one format by rasiing
        # 2 to the power of them and then converting them into binary.
        seenTeamsDict = {k: v for v, k in enumerate(seenTeams)}
        temp = []
        for x in self.gameList:
            tempi = []
            for z in x:
                if(z in seenTeams):
                    tempi.append(bin(2**seenTeamsDict[z])[2:].zfill(len(seenTeamsDict)))
                else:
                    tempi.append(z)
            temp.append(tempi)
        self.gameList = temp
        
    def removeQuotes(self, string):
        if (string.startswith('"') and string.endswith('"')) or (string.startswith("'") and string.endswith("'")):
            return string[1:-1]
        return string  
    
    def addGame(self, team1, score1, stats1, team2, score2, stats2):
        self.gameList.append([team1, score1, stats1, team2, score2, stats2])
        
    #give it two teams, the scores, and it will add it to the matrix
    def addScore(self, team1, team2, score1, score2):
        '''
        for a team in top array, the index in the array corrisponds to the matrix column there located in
        for a team in side array, the index in the array corrisponds to the matrix row there located in
        '''
        #team 1 score entry
        try:
            row = self.sideArray.index(team2)    

        except:
            self.sideArray.append(team2)
            row = self.sideArray.index(team2)    

        try:
            col = self.topArray.index(team1)
        except:
            self.topArray.append(team1)
            col = self.topArray.index(team1)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score1
                break
            counter += 1
        self.sc[row, col] = temp
        
        #team 2 score entry
        try:
            row = self.sideArray.index(team1)    
        except:
            self.sideArray.append(team1)
            row = self.sideArray.index(team1)    
            
        try:
            col = self.topArray.index(team2)
        except:
            self.topArray.append(team2)
            col = self.topArray.index(team2)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score2
                break
            counter += 1
        self.sc[row, col] = temp
    
    #returns the score(s) for match up
    def getScore(self, team1, team2, gameSelect = None):
        print(team1, team2)
        try:
            score1 = self.sc[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.sc[self.sideArray.index(team1), self.topArray.index(team2)]
            if (gameSelect == None):
                print(team1, score1)
                print(team2, score2)
            else:
                print(team1, score1[gameSelect])
                print(team2, score2[gameSelect])
        except:
            print('Invalid input of teams')
    
    def getGameList(self):
        return self.gameList
    
    #constructs a matrix of the avg score in a matchup
    def buildAvgMatrix(self): 
        for col in range(len(self.sc[:,0])):   #depth
            for row in range(len(self.sc[0, :])):  #width
                tempScore = self.sc[row, col]
                avgScore = 0.0
                count = 0.0
                for j in tempScore:
                    if (j != -1):
                        avgScore += j
                        count += 1
                    else:
                        break
                try:
                    avgScore = avgScore / count
                except:
                    avgScore = -1
                self.am[row, col] = avgScore
    
    #get the value of the avg score for a match up
    def getAvgScore(self, team1, team2):
        try:
            score1 = self.am[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.am[self.sideArray.index(team1), self.topArray.index(team2)]
            print(team1, score1)
            print(team2, score2)        
        except:
            print('Invalid input of teams')

In [10]:
import tensorflow as tf
import tflearn

tf.reset_default_graph()

# Parse the game files and grab the stats we want
# the indexes are put in the order: homeTeamName, homeTeamScore, homeTeamStats, awayName, awayScore, awayStats
stats = GameStats(3, 10, [5, 51, 55, 57, 58, 69, 74, 76], 6, 9, [8, 23, 27, 29, 30, 41, 46, 47])

gameList = stats.getGameList() # get the list of games
avgStats = dict()
np.set_printoptions(precision=4)  #IF you wanna remove this i recommend restarting the kernal

#getting rid of the strings
def removeQuotes(gameList):
    for row in gameList:
        for x in xrange(len(row)):
            #convert scores strings to float
            if (x == 1 or x == 4): row[x] = float(row[x])
            #convert arrays to floats
            if (x == 2 or x == 5): row[x] = map(float, row[x])
    return gameList

#adding up all the teams stats
gameList = removeQuotes(gameList)

#ERROR in the data, scroll down to see the outlire binary number
for row in gameList:
    print(row[2], row[5])

([1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 6.0, 0.0, 1.0, 3.0])
([1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0])
([1.0, 2.0, 1.0, 1.0, 5.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 5.0, 0.0, 0.0, 2.0])
([1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [1.0, 3.0, 0.0, 0.0, 7.0, 0.0, 0.0, 1.0])
([1.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0], [1.0, 3.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0])
([1.0, 2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0])
([2.0, 1.0, 0.0, 0.0, 4.0, 1.0, 0.0, 0.0], [2.0, 1.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0])
([1.0, 0.0, 0.0, 1.0, 6.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0])
([1.0, 2.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0], [1.0, 4.0, 0.0, 1.0, 4.0, 1.0, 0.0, 1.0])
([1.0, 0.0, 0.0, 1.0, 5.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 5.0, 0.0, 0.0, 2.0])
([1.0, 3.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0])
([1.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0], [1.0, 3.0, 0.0, 0.0, 5

In [13]:
for row in gameList:
    #home
    print("visitor", row[0])
    if avgStats.has_key(row[0]):
        print(avgStats[row[0]])
        avgStats[row[0]][0] = 0   #set the proir games to 0,because there set to right amount next line
        avgStats[row[0]] = np.sum([avgStats[row[0]], row[2]], axis=0)  #total stats + indivudal game stats
    else:
        avgStats[row[0]] = row[2]
    print(row[2])
    print(avgStats[row[0]])
    print('-------------')
    
    #away
    print("home", row[3])
    if avgStats.has_key(row[3]):
        
        print(avgStats[row[3]])
        avgStats[row[3]][0] = 0
        avgStats[row[3]] = np.sum([avgStats[row[3]], row[5]], axis=0)
    else:
        avgStats[row[3]] = row[5]
    print(row[5])
    print(avgStats[row[3]])
    print('-------------')




('visitor', '000000000000000000000000000001')
[  1.6200e+02   1.5926e+00   3.3951e-01   2.1605e-01   2.7099e+00
   3.0247e-01   4.3210e-02   3.8272e-01]
[0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0]
[ 0.      1.5926  0.3395  0.216   4.7099  0.3025  0.0432  0.3827]
-------------
('home', '000001000000000000000000000000')
[  1.6200e+02   1.5309e+00   1.8519e-01   3.7037e-01   3.1914e+00
   3.5185e-01   7.4074e-02   4.4444e-01]
[0, 1.0, 0.0, 0.0, 6.0, 0.0, 1.0, 3.0]
[ 0.      2.5309  0.1852  0.3704  9.1914  0.3519  1.0741  3.4444]
-------------
('visitor', '000000000000000000000000000010')
[  1.6200e+02   1.6605e+00   8.0247e-02   3.6420e-01   2.8457e+00
   3.8272e-01   8.6420e-02   5.7407e-01]
[0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
[ 0.      2.6605  0.0802  0.3642  3.8457  1.3827  0.0864  0.5741]
-------------
('home', '000000000000100000000000000000')
[  1.6200e+02   1.4383e+00   2.0370e-01   1.9753e-01   3.0309e+00
   4.2593e-01   8.6420e-02   2.3457e-01]
[0, 1.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0

In [14]:
#make the average..
keyList = avgStats.keys()    
for key in keyList:
    print(key)
    print(avgStats[key])  #print total stats
    for index in xrange(1, len(avgStats[key])):  #divide all stats by games played (162, 162)
        avgStats[key][index] = avgStats[key][index] / avgStats[key][0]  
    print(avgStats[key])  #print the avg stats
    print('-------------------------------------------')

000000000000000000000000001000
[ 162.      206.2654   31.1914   63.3889  498.0556   77.4753   13.0802
   58.358 ]
[  1.6200e+02   1.2732e+00   1.9254e-01   3.9129e-01   3.0744e+00
   4.7824e-01   8.0742e-02   3.6023e-01]
-------------------------------------------
000000000000000100000000000000
[ 161.      251.5528   26.1615   49.3043  467.8882   84.5217   14.087
   91.5652]
[  1.6100e+02   1.5624e+00   1.6249e-01   3.0624e-01   2.9061e+00
   5.2498e-01   8.7497e-02   5.6873e-01]
-------------------------------------------
000000000000000010000000000000
[ 162.      321.9753   18.1111   39.2407  455.7963   65.4012    9.0556
   79.4877]
[  1.6200e+02   1.9875e+00   1.1180e-01   2.4223e-01   2.8136e+00
   4.0371e-01   5.5898e-02   4.9066e-01]
-------------------------------------------
000000000000000001000000000000
[ 162.      252.5494   24.1481   65.4012  493.0247   67.4136   20.1235
   70.4321]
[  1.6200e+02   1.5589e+00   1.4906e-01   4.0371e-01   3.0434e+00
   4.1613e-01   1.2422e-01

In [15]:
#needs to be imported again
gameList = stats.getGameList() # get the list of games

#fixing 000000000000000000000000100000 because the hits number is bad (there are more problems in 2015 data)
avgStats['000000000000000000000000100000'][1] = 1.59   #some arbitrary value - i think the problem could be a cancelled game at game 97 or 96


#adding avg values to the list
for row in gameList:
    #home
    temp = avgStats[row[0]]   #using key to get avg stats array
    temp[0] = row[2][0]  #perserve the game num in the swap
    temp = np.around(temp, decimals=4)
    row[2] = temp
    #away
    temp = avgStats[row[3]]   #using key to get avg stats array
    temp[0] = row[5][0]
    temp = np.around(temp, decimals=4)
    row[5] = temp 

In [16]:
#the difference
gameList = removeQuotes(gameList)   #because i initialized gamestats above, must remove quote again
diffList = np.array([])

for row in gameList:
    diff = np.array([])    
    diff =  np.subtract(row[2], row[5])   #home stats - away stats (not including score)
    diff = np.append(diff, np.subtract(row[1], row[4]))   #home score - visitor score
    diff = np.delete(diff, 0)      #delete the game played column because subtracting those is usually 0    
    diffList = np.append(diffList, diff)
    print(diff)

    


[ 0.0621  0.1553 -0.1553 -0.4845 -0.0497 -0.031  -0.0621  1.    ]
[ 0.2236 -0.1243  0.1676 -0.1864 -0.0435  0.      0.3416 -2.    ]
[-0.2733 -0.1615 -0.0993 -0.3602 -0.0124  0.0186 -0.0622  3.    ]
[-0.4348  0.0559  0.0435 -0.0186  0.118   0.0248 -0.1739 -9.    ]
[ 0.1056  0.0559 -0.0622 -0.41   -0.0248  0.0062 -0.1304  1.    ]
[-0.1304  0.0559  0.1181  0.354   0.0993 -0.0248  0.1925 -1.    ]
[ 0.2236 -0.1243  0.1676 -0.1864 -0.0435  0.      0.3416 -2.    ]
[ 0.0808  0.0249  0.0124 -0.4596  0.     -0.0124 -0.0124  1.    ]
[-0.1553  0.0434  0.0808 -0.3478  0.0621 -0.0311  0.031  -5.    ]
[-0.0785 -0.002  -0.0835 -0.5119 -0.0148 -0.0689 -0.1275 -1.    ]
[ 0.2919 -0.0311 -0.1677 -1.0559  0.0683 -0.0124 -0.0435  4.    ]
[ -1.9250e-01   6.2000e-03  -1.2420e-01  -5.7760e-01   1.3670e-01
   0.0000e+00   6.2000e-03  -9.0000e+00]
[ -3.1050e-01   4.9700e-02   0.0000e+00  -6.5210e-01   9.3200e-02
  -6.2000e-03   1.4290e-01  -1.5000e+01]
[-0.4348  0.0559  0.0435 -0.0186  0.118   0.0248 -0.1739 -5.